<a href="https://colab.research.google.com/github/panghanwu/machine_learning_Elwing/blob/main/RNN%26LSTM%26GRU_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

不推薦RNN系列，無法平行運算，效果也沒有特別好。

In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import glob
import pandas as pd

def read(path):
  with open(path, 'r', encoding='utf-8') as f:
    content = f.read()
  return content

# get path
dn = os.path.dirname(dataset)
pattern = os.path.join(dn, 'aclImdb', 'train', 'pos', '*')
pos = glob.glob(pattern)
pattern = os.path.join(dn, 'aclImdb', 'train', 'neg', '*')
neg = glob.glob(pattern)
sentiments = [1]*len(pos) + [0]*len(neg)

contents = map(read, pos+neg)


def get_data(t):
  dn = os.path.dirname(dataset)
  pattern = os.path.join(dn, "aclImdb", t, "pos", "*.txt")
  pos = glob.glob(pattern)
  pattern = os.path.join(dn, "aclImdb", t, "neg", "*.txt")
  neg = glob.glob(pattern)
  sentiments = [1] * len(pos) + [0] * len(neg)
  contents = map(read, pos + neg)
  df = pd.DataFrame({
      "contents":contents,
      "sentiment":sentiments
  })
  return df

train_df = get_data('train')
test_df = get_data('test')

# parameter
TOK = 3000
LEN = 512
EM = 128

tok = Tokenizer(num_words=TOK)
tok.fit_on_texts(train_df['contents'])

x_train_seq = tok.texts_to_sequences(train_df['contents'])
x_test_seq = tok.texts_to_sequences(test_df['contents'])

x_train_pad = pad_sequences(x_train_seq, LEN)
x_test_pad = pad_sequences(x_test_seq, LEN)


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import SimpleRNN
"""RNN""" 
layers = [
      Embedding(TOK+1, EM, mask_zero=True, input_length=LEN),
      SimpleRNN(64),
      Dense(2, activation="softmax")  
]

model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 128)          384128    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 64)                12352     
_________________________________________________________________
dense (Dense)                (None, 2)                 130       
Total params: 396,610
Trainable params: 396,610
Non-trainable params: 0
_________________________________________________________________


In [4]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer=Adam(),
       metrics=["accuracy"])

y_train = train_df["sentiment"]
y_test = test_df["sentiment"]

callbacks = [
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad, 
     y_train,
     batch_size=100,
     epochs=50,
     validation_split=0.1,
     callbacks=callbacks)

Epoch 1/50
225/225 [==============================] - 68s 302ms/step - loss: 0.5707 - accuracy: 0.6868 - val_loss: 0.7486 - val_accuracy: 0.5468
Epoch 2/50
225/225 [==============================] - 68s 304ms/step - loss: 0.3656 - accuracy: 0.8447 - val_loss: 0.6132 - val_accuracy: 0.6712
Epoch 3/50
225/225 [==============================] - 68s 302ms/step - loss: 0.3480 - accuracy: 0.8536 - val_loss: 0.4332 - val_accuracy: 0.8048
Epoch 4/50
225/225 [==============================] - 67s 297ms/step - loss: 0.3364 - accuracy: 0.8498 - val_loss: 0.4595 - val_accuracy: 0.8124
Epoch 5/50
225/225 [==============================] - 67s 298ms/step - loss: 0.2678 - accuracy: 0.8862 - val_loss: 0.6593 - val_accuracy: 0.6612
Epoch 6/50
225/225 [==============================] - 67s 298ms/step - loss: 0.2481 - accuracy: 0.8958 - val_loss: 0.6519 - val_accuracy: 0.7524
Epoch 7/50
225/225 [==============================] - 67s 296ms/step - loss: 0.1075 - accuracy: 0.9636 - val_loss: 0.8543 - val_ac

In [5]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 36s 46ms/step - loss: 0.4349 - accuracy: 0.8245


[0.434914767742157, 0.8245199918746948]

In [6]:
from tensorflow.keras.layers import LSTM
"""LSTM""" 
layers = [
      Embedding(TOK+1, EM, mask_zero=True, input_length=LEN),
      LSTM(64),
      Dense(2, activation="softmax")  
]

model = Sequential(layers)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 512, 128)          384128    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 433,666
Trainable params: 433,666
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer=Adam(),
       metrics=["accuracy"])

y_train = train_df["sentiment"]
y_test = test_df["sentiment"]

callbacks = [
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad, 
     y_train,
     batch_size=100,
     epochs=50,
     validation_split=0.1,
     callbacks=callbacks)

Epoch 1/50
225/225 [==============================] - 185s 823ms/step - loss: 0.4576 - accuracy: 0.7716 - val_loss: 0.5513 - val_accuracy: 0.7596
Epoch 2/50
225/225 [==============================] - 180s 802ms/step - loss: 0.2864 - accuracy: 0.8828 - val_loss: 0.3805 - val_accuracy: 0.8208
Epoch 3/50
225/225 [==============================] - 181s 803ms/step - loss: 0.2416 - accuracy: 0.9055 - val_loss: 0.4617 - val_accuracy: 0.8304
Epoch 4/50
225/225 [==============================] - 185s 823ms/step - loss: 0.2071 - accuracy: 0.9188 - val_loss: 0.5104 - val_accuracy: 0.7976
Epoch 5/50
225/225 [==============================] - 181s 803ms/step - loss: 0.1735 - accuracy: 0.9335 - val_loss: 0.3881 - val_accuracy: 0.8548
Epoch 6/50
225/225 [==============================] - 182s 808ms/step - loss: 0.1752 - accuracy: 0.9312 - val_loss: 0.2761 - val_accuracy: 0.8972
Epoch 7/50
225/225 [==============================] - 186s 828ms/step - loss: 0.1809 - accuracy: 0.9290 - val_loss: 0.3777 -

In [8]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 66s 85ms/step - loss: 0.3439 - accuracy: 0.8697


[0.3439391553401947, 0.8696799874305725]

In [9]:
from tensorflow.keras.layers import GRU
"""GRU""" 
layers = [
      Embedding(TOK+1, EM, mask_zero=True, input_length=LEN),
      GRU(64),
      Dense(2, activation="softmax")  
]

model = Sequential(layers)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 512, 128)          384128    
_________________________________________________________________
gru (GRU)                    (None, 64)                37248     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 421,506
Trainable params: 421,506
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer=Adam(),
       metrics=["accuracy"])

y_train = train_df["sentiment"]
y_test = test_df["sentiment"]

callbacks = [
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad, 
     y_train,
     batch_size=100,
     epochs=50,
     validation_split=0.1,
     callbacks=callbacks)

Epoch 1/50
225/225 [==============================] - 118s 524ms/step - loss: 0.4571 - accuracy: 0.7684 - val_loss: 0.4868 - val_accuracy: 0.7840
Epoch 2/50
225/225 [==============================] - 116s 517ms/step - loss: 0.2798 - accuracy: 0.8872 - val_loss: 0.3588 - val_accuracy: 0.8536
Epoch 3/50
225/225 [==============================] - 120s 533ms/step - loss: 0.2310 - accuracy: 0.9088 - val_loss: 0.4324 - val_accuracy: 0.8276
Epoch 4/50
225/225 [==============================] - 117s 522ms/step - loss: 0.1968 - accuracy: 0.9238 - val_loss: 0.3263 - val_accuracy: 0.8812
Epoch 5/50
225/225 [==============================] - 118s 523ms/step - loss: 0.1729 - accuracy: 0.9344 - val_loss: 0.3721 - val_accuracy: 0.8696
Epoch 6/50
225/225 [==============================] - 118s 523ms/step - loss: 0.1428 - accuracy: 0.9476 - val_loss: 0.5267 - val_accuracy: 0.8304
Epoch 7/50
225/225 [==============================] - 118s 523ms/step - loss: 0.1235 - accuracy: 0.9557 - val_loss: 0.4721 -

In [11]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 58s 74ms/step - loss: 0.3094 - accuracy: 0.8734


[0.309406042098999, 0.8733999729156494]